## 定义模型

In [9]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.env_util import *
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def get_completion(prompt):
    message = [{"role": "user", "content": prompt}]
    model = ChatOpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        model_name=os.getenv("MODEL_NAME"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=0.0,
    )
    return model.invoke(message)

get_completion("你是谁")

python-dotenv could not parse statement starting at line 6


AIMessage(content='\n我是基于人工智能技术的语言模型助手，由智谱 AI 公司开发。我的核心功能是通过自然语言对话，为用户提供信息查询、知识解答、创意生成、学习辅助等服务。以下是我的主要特点：\n\n1. **知识库**：截至2023年10月，我接受了涵盖科技、历史、文化、生活等领域的海量数据训练，能提供广泛的知识支持。\n\n2. **交互能力**：\n   - 支持多轮对话\n   - 可处理中英文混合输入\n   - 能理解上下文逻辑关系\n\n3. **应用场景**：\n   - 学术研究辅助\n   - 商业决策参考\n   - 日常问题解答\n   - 创意内容生成\n\n4. **技术特性**：\n   - 基于Transformer架构\n   - 采用1750亿参数的GPT-4架构\n   - 支持上下文记忆（约4096个token）\n\n5. **使用限制**：\n   - 知识截止日期为2023年10月\n   - 无法进行实时网络搜索\n   - 不具备物理世界交互能力\n   - 需要用户主动提供背景信息\n\n6. **隐私保护**：\n   - 不存储对话记录\n   - 不收集个人身份信息\n   - 符合GDPR等数据保护规范\n\n如果您有具体问题或需要帮助完成某项任务，可以随时告诉我，我会尽力提供专业、准确的解答。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 520, 'prompt_tokens': 10, 'total_tokens': 530, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 218, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'THUDM/GLM-Z1-32B-0414', 'system_fingerprint': '', 'id': '01968a5e

## Prompt 模板

In [10]:
prompt_template = """
你是一个问答机器人，你的任务是根据下述给定的已知信息回答用户问题。如果你不知道答案，就回答不知道，不要胡编乱造。

已知信息：
{context}

用户问题：
{query}

如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，就回答不知道，不要胡编乱造。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
"""

def build_prompt(prompt_template, **kwargs):
    """
    自定义参数渲染 Prompt 模板
    """
    inputs = {}
    for k, v in kwargs.items():
        if isinstance(v, list) and all(isinstance(elem, str) for elem in v):
            val = "\n\n".join(v)
        else:
            val = v
        inputs[k] = val
    return prompt_template.format(**inputs)


## 向量间相似度的计算

余弦相似度通过向量点积与模长乘积的比值计算，公式如下： 

$$
\cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}
$$

L2 距离是向量对应元素差的平方和的平方根，公式如下：

$$
d(\mathbf{A}, \mathbf{B}) = \sqrt{\sum_{i=1}^{n} (A_i - B_i)^2}
$$

In [11]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(A, B):
    """ 余弦相似度（越大越相似） """
    return dot(A, B)/(norm(A)*norm(B))

def l2_distance(A, B):
    """ 欧式距离（越小越相似） """
    x = np.asarray(A)
    y = np.asarray(B)
    return norm(x-y)

## 嵌入（Embedding）模型

[嵌入模型库](https://www.modelscope.cn/models?page=1&tasks=sentence-embedding&type=nlp)

标准

- **找需求相关的语料库来进行文本向量转换测试，进行评估**
- 向量模型的精确度直接影响相似度检索的文档召回率
- 大多数场景下，开源的嵌入模型使用效果都一般，要进行检索召回率，建议对模型进行微调
- 嵌入模型的维度越高，表示特征细节提取越丰富

In [12]:
from typing import List
from langchain_openai import OpenAIEmbeddings
from volcenginesdkarkruntime import Ark

def get_embeddings(texts: List[str]):
    """ OpenAI Embeddings """
    embedding_model = OpenAIEmbeddings(
        api_key=os.getenv('OPENAI_API_KEY'),
        base_url=os.getenv('OPENAI_BASE_URL'),
        model=os.getenv('EMBEDDING_MODEL_NAME')
    )
    return embedding_model.embed_documents(texts)

def get_ark_embeddings(texts: List[str]):
    client = Ark(api_key=os.getenv('ARK_API_KEY'))
    return client.embeddings.create(
        model="doubao-embedding-large-text-240915",
        input=texts
    )

# def get_embeddings(texts, model="text-embedding-ada-002", dimensions=None):
#     '''封装 OpenAI 的 Embedding 模型接口'''
#     if model == "text-embedding-ada-002":
#         dimensions = None
#     if dimensions:
#         data = client.embeddings.create(
#             input=texts, model=model, dimensions=dimensions).data
#     else:
#         data = client.embeddings.create(input=texts, model=model).data
#     return [x.embedding for x in data]

test_texts = ['吃完海鲜可以喝牛奶吗？']
vec = get_embeddings(test_texts)[0]
print(f'Total Dimension: {len(vec)}')
print(f'Fist 10 Dimensions: {vec[:10]}')

vec = get_ark_embeddings(test_texts).data[0].embedding
print(f'\n[ARK] Total Dimension: {len(vec)}')
print(f'[ARK] Fist 10 Dimensions: {vec[:10]}')

BadRequestError: Error code: 400 - {'code': 20012, 'message': 'Model does not exist. Please check it carefully.', 'data': None}

In [16]:
query = "国际争端"
# 前两条为国际争端
documents = [
    "联合国就苏丹达尔富地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本歧阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验"
]

# query_vector = get_embeddings([query])[0]
# doc_vectors = get_embeddings(documents)

# 实测 doubao Embedding 模型效果更好
query_vector = get_ark_embeddings([query]).data[0].embedding
doc_vectors = [doc_vector.embedding for doc_vector in get_ark_embeddings(documents).data]

# 余弦距离越大表示越相似
print(f'query 与自己的余弦相似度为：{cosine_similarity(query_vector, query_vector)}')
print('query 与文档的余弦相似度为：')
for doc_vector in doc_vectors:
    print(f'{cosine_similarity(query_vector, doc_vector)}')

print('*' * 80)

# L2距离（欧式距离）越小表示越相似
print(f'query 与自己的L2距离为：{l2_distance(query_vector, query_vector)}')
print('query 与文档的L2距离为：')
for doc_vector in doc_vectors:
    print(f'{l2_distance(query_vector, doc_vector)}')

query 与自己的余弦相似度为：1.0
query 与文档的余弦相似度为：
0.8854540946810185
0.8605012890827118
0.80269178686373
0.7593635176801089
0.7953524476835953
********************************************************************************
query 与自己的L2距离为：0.0
query 与文档的L2距离为：
63.82647284306631
70.09578368205798
82.47880162653664
91.39691180339793
83.86019628342954


## 向量数据库 Chroma

<div class="alert alert-info">
官方文档：https://docs.trychroma.com/docs/overview/introduction<br>
<b>扩展阅读：</b><a herf="https://guangzhengli.com/blog/zh/vector-database">https://guangzhengli.com/blog/zh/vector-database</a>
</div>

<div class="alert alert-success">
<b>澄清几个关键概念：</b><ul>
    <li>向量数据库的意义是快速的检索；</li>
    <li>向量数据库本身不生成向量，向量是由 Embedding 模型产生的；</li>
    <li>向量数据库与传统的关系型数据库是互补的，不是替代关系，在实际应用中根据实际需求经常同时使用。</li>
</ul>
</div>

In [17]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

def extract_text_from_pdf(filename, page_numbers=None, min_line_length=1):
    """Extract text from a PDF file.
    Args:
        filename (str): Path to the PDF file.
        page_numbers (list[int], optional): List of page numbers to extract. If None, extract all pages.
        min_line_length (int, optional): Minimum length of a line to be included in the output.
    Returns:
        str: Extracted text.
    """
    paragraphs = []
    buffer, full_text = '', ''
    # 提取全部文本
    for i, page_layout in enumerate(extract_pages(filename)):
        if page_numbers is not None and i not in page_numbers:
            continue
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                full_text += element.get_text() + '\n'
    # 按空行分隔，将文本重新组织成段落
    for line in full_text.split('\n'):
        if len(line) >= min_line_length:
            buffer += (' ' + line) if not line.endswith('-') else line.strip('-')
        elif buffer:
            paragraphs.append(buffer.strip())
            buffer = ''
    if buffer:
        paragraphs.append(buffer.strip())
    return paragraphs  

# pdfminer 的效果不太好，这里只演示效果，实际用 markdown 文件
paragraphs = extract_text_from_pdf("file/医学史.pdf", min_line_length=45)
for paragraph in paragraphs[:3]:
    print(paragraph + '\n')   

医学的发展不能脱离它所处的时代。医学思想和实践来自与之相适应的知识环境，同时又为拓展和丰富人类的知识

贡献力量。从原始社会到现在几千年的发展历程，医学的发展道路艰难曲折，不仅囊括了医学的各门学科，而且还

涉及丰富多彩的人类医疗卫生活动。医学的发展凝聚着一代又一代先行者的心血和智慧，它既是人类对自身疾病与



In [18]:
# 为了演示方便，我们只取两页（第一章/Page 3、Page 4）
# paragraphs = extract_text_from_pdf(
#     "file/医学史.pdf",
#     page_numbers=[3, 3],
#     min_line_length=10
# )

# for paragraph in paragraphs:
#     print(paragraph + '\n')   

## 分割/检索 markdown 文件

**文本分割的粒度**  
1. 粒度太大可能导致检索不精准，粒度太小可能导致信息不全面
2. 问题的答案可能跨越两个片段

LangChain 文本分割的策略：
1. **基于字符的分割（`CharacterTextSplitter`）**

    按照指定的字符进行分割，默认使用换行符和空格作为分隔符。通过设置 `chunk_size` 和 `chunk_overlap` 参数，可以控制每个文本块的大小和重叠部分。

    ```python
    from langchain.text_splitter import CharacterTextSplitter

    text = "这是一段示例文本，用于演示文本分割的策略。"
    text_splitter = CharacterTextSplitter(
        separator=" ",
        chunk_size=10,
        chunk_overlap=2
    )
    chunks = text_splitter.split_text(text)
    print(chunks)
    ```

2. **基于 token 的分割（`TokenTextSplitter`）**

    `TokenTextSplitter` 基于 `token` 进行分割，而不是字符。标记是自然语言处理中对文本进行分词后的基本单元。通过设置 `chunk_size 和 `chunk_overlap` 参数，可以控制每个文本块包含的标记数量。


3. **按文档结构分割（`MarkdownTextSplitter`、`HtmlTextSplitter`）**

    `MarkdownTextSplitter` 专门用于分割 Markdown 文档，它会根据 Markdown 的语法结构（如标题、列表等）进行分割，确保分割后的文本块具有一定的语义完整性。  
    `HtmlTextSplitter` 用于分割 HTML 文档，它会根据 HTML 的标签结构进行分割，去除 HTML 标签，只保留文本内容，并将文本分割成合适的块。

4. **递归分割（`RecursiveCharacterTextSplitter`）**

    它会尝试按照多个分隔符进行分割，优先使用较长的分隔符，以确保分割后的文本块具有较好的语义完整性。如果使用较长的分隔符无法满足 chunk_size 的要求，则会尝试使用较短的分隔符。

    ```python
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    text = "这是一段示例文本，用于演示递归文本分割的策略。"
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["。", "，", " "],
        chunk_size=10,
        chunk_overlap=2
    )
    chunks = text_splitter.split_text(text)
    print(chunks)
    ```

5. **基于语义分割**

    1. 根据句子、段落或主题等有语义的单元进行分割；
    2. 为每个片段创建 Embeddig；
    3. 对相邻的片段计算余弦相似度，选择相似度最高的两个片段进行合并，直到余弦相似度显著下降位置。

    > 或者使用早期预训练模型（如 BERT）中用于增强句子级语义理解的 NSP 任务（Next Sentence Prediction，下一句预测任务） 方法。

6. **基于 LLM 分割**

    通过 Prompt + LLM 的方式，将文本分割成多个片段，每个片段包含一个语义单元。这种方法将保证较高的语义准确性，因为 LLM 可以更好地理解文本的语义。

---

在实际应用中，选择合适的文本分割器应根据具体需求进行。例如：
- 对于简单文本，可以选择 `CharacterTextSplitter`。
- 处理长文本或需要上下文信息的场合，推荐使用` RecursiveCharacterTextSplitter` 或 `TokenTextSplitter`。
- 中文文章推荐用 `RecursiveCharacterTextSplitter`
- 处理 `Markdown` 文档时，`MarkdownTextSplitter` 是最佳选择，而对于`LaTeX`文档，则应使用`LatexTextSplitter`。

In [19]:
from nltk.tokenize import sent_tokenize


# chunk_size 一般根据文档内容或大小来设置
# overlap_size 一般设置 chunk_size 大小的10%-20%之间
def split_text(paragraphs, chunk_size=300, overlap_size=100):
    '''按指定 chunk_size 和 overlap_size 交叠割文本'''
    sentences = [s.strip() for p in paragraphs for s in sent_tokenize(p)]
    chunks = []
    i = 0
    while i < len(sentences):
        chunk = sentences[i]
        overlap = ''
        prev_len = 0
        prev = i - 1
        # 向前计算重叠部分
        while prev >= 0 and len(sentences[prev])+len(overlap) <= overlap_size:
            overlap = sentences[prev] + ' ' + overlap
            prev -= 1
        chunk = overlap+chunk
        next = i + 1
        # 向后计算当前chunk
        while next < len(sentences) and len(sentences[next])+len(chunk) <= chunk_size:
            chunk = chunk + ' ' + sentences[next]
            next += 1
        chunks.append(chunk)
        i = next
    return chunks

<div class="alert alert-info">
此处 sent_tokenize 为针对英文的实现，针对中文的实现请参考 <a herf='./chinese_utils.py'>chinese_utils.py</a>
</div>

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def read_markdown_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"错误：未找到文件 {file_path}。")
        return None
    except Exception as e:
        print(f"错误：读取文件时发生未知错误 {e}。")
        return None

file_path = os.path.abspath(os.path.join(os.getcwd(), 'file/医学史.md'))
markdown_content = read_markdown_file(file_path)



docs = []
if markdown_content:
    # 创建 Markdown 文本分割器
    # markdown_splitter = MarkdownTextSplitter(chunk_size=512, chunk_overlap=250)
    markdown_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=256)

    # 分割 Markdown 内容为多个 Document
    docs = markdown_splitter.create_documents([markdown_content])

    # 打印每个 Document 的内容
    for i, doc in enumerate(docs[:2]):
        print(f"************************** Document {i + 1}: *********************************")
        print(doc.page_content)

************************** Document 1: *********************************
医学的发展不能脱离它所处的时代。医学思想和实践来自与之相适应的知识环境，同时又为拓展和丰富人类的知识贡献力量。从原始社会到现在几千年的发展历程，医学的发展道路艰难曲折，不仅囊括了医学的各门学科，而且还涉及丰富多彩的人类医疗卫生活动。医学的发展凝聚着一代又一代先行者的心血和智慧，它既是人类对自身疾病与健康及其关系的认识史，也是一部伴随着社会生产发展，由经验到科学，由低级到高级、由单一到综合逐渐进化的发展史，既是科学技术进步的一个缩形，也是人类文化史的一个重要组成。

## 一、古代医学
受古代生产力及科学技术水平的限制，古代医学知识多来源于医疗实践经验的积累，夹杂着唯心主义和迷信思想。宗教和文学在一定程度上促进了古代医学的发展。在数千年的历史长河中，古代医学经历了漫长的发展历程。

### （一）史前医学
在漫长的 300 至万年前的原始社会中，原始人类在同疾病斗争的过程中逐步认识了各种植物、动物和矿物等的药效，开辟了以经验为起源的各项医药活动。在原始社会末期，已有了断肢术、阉割术，穿颅术、剖宫产术等外科手术和相应的外伤治疗外用药物。这些进步，也与社会生产工具的发明和改进密切相关。
************************** Document 2: *********************************
### （一）史前医学
在漫长的 300 至万年前的原始社会中，原始人类在同疾病斗争的过程中逐步认识了各种植物、动物和矿物等的药效，开辟了以经验为起源的各项医药活动。在原始社会末期，已有了断肢术、阉割术，穿颅术、剖宫产术等外科手术和相应的外伤治疗外用药物。这些进步，也与社会生产工具的发明和改进密切相关。

由于原始社会的生产力低下，人们不能正确理解疾病和死亡，认为疾病是由一种超自然的力量所形成和主宰的，认为自然界的一切现象都由一种超自然的实体控制，从而产生了万物有灵的观念：山神、河怪、精灵、疯魔等都可导致人类疾病。在这些认识支配下，产生了招魂、驱鬼、敬神、祷告等病方法，产生了祭司和巫师，也产生了图腾崇拜，医学被打上了原始宗教的烙印。

### （二）古代东方医学


In [46]:
import chromadb
from chromadb.config import Settings

class MyVectorDBConnector:
    def __init__(self, collection_name, embedding_fn, batch_size=None):
        # 内存模式
        chroma_client = chromadb.Client(Settings(allow_reset=True))
        # 数据持久化
        # chroma_client = chromadb.PersistentClient(path="./chroma")

        # NOTE: 清空数据，为了演示，实际不需要每次 reset()，并且是不可逆的
        chroma_client.reset()

        # 创建一个 collection
        self.collection = chroma_client.get_or_create_collection(
            name=collection_name, 
            metadata={"hnsw:space": "cosine"} # l2 is the default
        )
        self.embedding_fn = embedding_fn

    def add_documents(self, documents, batch_size=None):
        '''向 collection 中添加文档与向量'''
        n = len(documents)
        if batch_size is None:
            self.collection.add(
                embeddings=self.embedding_fn(documents),        # 每个文档的向量
                documents=documents,                            # 文档的原文
                ids=[f"id{i}" for i in range(n)]   # 每个文档的 id
            )
        else:
            for i in range(0, n, batch_size):
                end_idx = min(i + batch_size, n)
                self.collection.add(
                    embeddings=self.embedding_fn(documents[i:end_idx]),        # 每个文档的向量
                    documents=documents[i:end_idx],                            # 文档的原文
                    ids=[f"id{i}" for i in range(i, end_idx)]   # 每个文档的 id
                )
        print(f"\n 😁 添加{n}份文档成功 😁\n")

    def search(self, query, top_n):
        '''检索向量数据库'''
        results = self.collection.query(
            query_embeddings=self.embedding_fn([query]),
            n_results=top_n
        )
        return results

**NOTE:** 这里如果提示 `sqlite` 版本过低，可以安装 `pysqlite3-binary`，然后在虚拟环境的 `chromadb/__init__.py` 中添加如下代码：
```python
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
```

在 Chroma 里，`collection.query` 返回结果中的 `distances` 表示查询向量和集合中返回的文档向量之间的距离。一般而言，这个值越小代表两个向量越相似。
1. 余弦距离（`distance_metric='cosine'`）
含义：余弦距离基于余弦相似度得出，公式为 `余弦距离 = 1 - 余弦相似度`。余弦相似度衡量的是两个向量夹角的余弦值，范围是 [-1, 1]。当两个向量方向完全相同时，余弦相似度为 1，余弦距离为 0；当两个向量方向完全相反时，余弦相似度为 -1，余弦距离为 2。
评判标准：余弦距离越接近 0，意味着查询向量和文档向量的方向越相近，也就是两个向量越相似。所以在使用余弦距离时，`distances` 的值越小越好。

2. 欧几里得距离（`distance_metric='l2'`）
含义：欧几里得距离指的是 n 维空间中两个点之间的直线距离。
评判标准：欧几里得距离为 0 时，表示两个向量完全相同；值越大，表明两个向量在空间中的距离越远，相似度越低。所以使用欧几里得距离时，同样是 `distances` 的值越小越好。

3. 内积距离（`distance_metric='ip'`）
含义：内积距离基于向量的内积来计算。内积反映了两个向量在方向上的一致性和长度的乘积。
评判标准：一般来说，内积值越大，两个向量越相似，但在 Chroma 中使用内积作为距离度量时，通常会对其进行转换，使得距离值越小表示越相似。

在 Chroma 中，`collection.query` 返回的结果默认是按照 距离从小到大排序的，因此 `distances` 越小的结果会排在越前面。

In [47]:
# 提取文档的文本内容
document_texts = [doc.page_content for doc in docs]
# 创建一个向量数据库对象
vector_db = MyVectorDBConnector("demo", get_embeddings)
# 向向量数据库中添加文档
vector_db.add_documents(document_texts)

user_query = """17世纪初，由于什么的应用，使生命科学的研究步入科学轨道"""
results = vector_db.search(user_query, 5)

for idx, para in enumerate(results['documents'][0]):
    print(f"**************** 😁 rank:{idx +1} | distance: {results['distances'][0][idx]} *****************")
    print(para+"\n")


 😁 添加60份文档成功 😁

**************** 😁 rank:1 | distance: 0.33148616552352905 *****************
1953年，美国分子生物学家洪森 (James Dewey Watson.1928-)和英国物理学家克里克 (FrancisHarry Compton Crick,1916--2004）以及英国物理学家威尔金斯 (Maurice Hugh Frederick Wilkins, 1916--2004) 发现并阐明了 DNA 分子的双螺旋结构，奠定了分子生物学的基础。他们三人分获 1962年诺贝尔生理学或医学奖。1965年，我国科学家在世界上首次用化学方法合成了牛胰岛素；之后，美国科学家也合成了含有206个核苷酸的 DNA 大分子。70年代发现了逆转录酶和限制性内切酶，促进了基因工程的发展。80年代初，临床开始应用基因工程治疗疾病，如用单克隆技术治疗癌症。人们在发酵工业中大量生产胰岛素，保证了临床用药的需要，降低了成本，减轻了病人的经济负担。

（2）医学遗传学的发展：

医学遗传学是研究人类疾病与遗传的关系，研究人类遗传病形成的机制和遗传方式，以及遗传病的诊断、治疗、预后、复发危险和预防的科学，是医学与遗传学相结合的边缘学科。

**************** 😁 rank:2 | distance: 0.33584147691726685 *****************
显微镜的应用为 19 世纪细胞学的建立打下了良好的基础。

#### 3．医学三学派的成熟

在17世纪，由于物理学、化学和生物学的进步，使一些学者主张以单一学科的理论来解释生命现象和病理现象，出现三个学派。

（1）物理学派：代表人物是法国数学家、物理学家笛卡尔。他认为：“宇宙是一个庞大的机械，人的身体也是一部精细的机械，从宏观到微观，所有物体无一不可用机械原理来阐明。”身体的一切疼痛、恐怖表现都是机械的反应。伽利略的学生波累利认为肌肉运动是一种力学原理，人心脏的搏动、胃肠 运动都符合力学原理。他甚至认为胃的消化功能就是摩擦力作用的结果。

**************** 😁 rank:3 | distance: 0.3737780451774597 *****************
17世纪的医学进步得

## 基于向量检索的RAG

In [48]:
class RAG_Bot:
    def __init__(self, vector_db, llm_api, n_results=3):
        self.vector_db = vector_db
        self.llm_api = llm_api
        self.n_results = n_results

    def chat(self, user_query):
        # 1. 检索
        search_results = self.vector_db.search(user_query, self.n_results)

        # 2. 构建 Prompt
        prompt = build_prompt(
            prompt_template, context=search_results['documents'][0], query=user_query)

        # 3. 调用 LLM
        response = self.llm_api(prompt)
        return response

In [49]:
# 创建一个RAG机器人
bot = RAG_Bot(vector_db,llm_api=get_completion)

user_query = "17世纪初，由于什么的应用，使生命科学的研究步入科学轨道?"

response = bot.chat(user_query)
print(response.content)



量度的应用。


## 检索后排序（rerank）

从向量数据库里召回后，有时候相关性高的文档不一定排在前面，需要根据相关性排序。

In [50]:
user_query = """17世纪初，由于什么的应用，使生命科学的研究步入科学轨道"""
results = vector_db.search(user_query, 5)

# 这里可以看出，最相关的文档为第3个文档
for idx, para in enumerate(results['documents'][0]):
    print(f"**************** rank:{idx +1} | distance: {results['distances'][0][idx]} *****************")
    print(para+"\n")

**************** rank:1 | distance: 0.33171558380126953 *****************
1953年，美国分子生物学家洪森 (James Dewey Watson.1928-)和英国物理学家克里克 (FrancisHarry Compton Crick,1916--2004）以及英国物理学家威尔金斯 (Maurice Hugh Frederick Wilkins, 1916--2004) 发现并阐明了 DNA 分子的双螺旋结构，奠定了分子生物学的基础。他们三人分获 1962年诺贝尔生理学或医学奖。1965年，我国科学家在世界上首次用化学方法合成了牛胰岛素；之后，美国科学家也合成了含有206个核苷酸的 DNA 大分子。70年代发现了逆转录酶和限制性内切酶，促进了基因工程的发展。80年代初，临床开始应用基因工程治疗疾病，如用单克隆技术治疗癌症。人们在发酵工业中大量生产胰岛素，保证了临床用药的需要，降低了成本，减轻了病人的经济负担。

（2）医学遗传学的发展：

医学遗传学是研究人类疾病与遗传的关系，研究人类遗传病形成的机制和遗传方式，以及遗传病的诊断、治疗、预后、复发危险和预防的科学，是医学与遗传学相结合的边缘学科。

**************** rank:2 | distance: 0.335904598236084 *****************
显微镜的应用为 19 世纪细胞学的建立打下了良好的基础。

#### 3．医学三学派的成熟

在17世纪，由于物理学、化学和生物学的进步，使一些学者主张以单一学科的理论来解释生命现象和病理现象，出现三个学派。

（1）物理学派：代表人物是法国数学家、物理学家笛卡尔。他认为：“宇宙是一个庞大的机械，人的身体也是一部精细的机械，从宏观到微观，所有物体无一不可用机械原理来阐明。”身体的一切疼痛、恐怖表现都是机械的反应。伽利略的学生波累利认为肌肉运动是一种力学原理，人心脏的搏动、胃肠 运动都符合力学原理。他甚至认为胃的消化功能就是摩擦力作用的结果。

**************** rank:3 | distance: 0.37386780977249146 *****************
17世纪的医学进步得益于伽利略（Galileo Galilei. 1

In [57]:
import requests

class Rerank:
    def __init__(self, model: str, base_url: str, api_key: str):
        self.rerank_model = model
        self.base_url = base_url
        self.api_key = api_key
        print(f'rerank 初始化成功：{self.rerank_model}, {self.base_url}, {self.api_key}')

    def rerank_cloud(self, results: List[str], query: str, k=10) -> List[str]:
        """
        使用云端重排序模型模型对检索结果进行重排序

        :param results: 原始检索结果
        :param query: 查询
        :param k: 返回的top-k结果数
        :return: 重排序后的结果
        """
        # texts = [item.page_content for item in results]

        url = self.base_url

        payload = {
            "model": self.rerank_model,
            "query": query,
            "documents": results,
            "top_n": k,
            "return_documents": False,
            "max_chunks_per_doc": 512,
            "overlap_tokens": 256
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        response = requests.request("POST", url, json=payload, headers=headers)
        try:
            if response.status_code == 200:
                response = response.json()
                indices = [item['index'] for item in response['results']]
                return [results[i] for i in indices]
            else:
                print(f"❌ {response.json()}")
        except:
            print(f'Error:network error status_code={response.status_code}')


In [68]:
load_dotenv()
rerank = Rerank(
    model=os.getenv('RERANK_MODEL_NAME'),
    base_url="https://api.siliconflow.cn/v1/rerank",
    api_key=os.getenv('OPENAI_API_KEY')
)

rerank_results = rerank.rerank_cloud(results['documents'][0], user_query, k=3)
# 这里可以看出，rerank后，最相关的文档为第1个文档
for idx, para in enumerate(rerank_results):
    print(f"**************** rank:{idx +1} *****************")
    print(para+"\n")

rerank 初始化成功：BAAI/bge-reranker-v2-m3, https://api.siliconflow.cn/v1/rerank, sk-hybehttizlquaobtbilikijqmuuyzxizjhkfqqlpkkvcvojw
rerank cloud success
**************** rank:1 *****************
17世纪的医学进步得益于伽利略（Galileo Galilei. 1564-1642)和刻卜勒（Jobannes Kepler,1571-1630）等一批杰出科学家的成就。例如帕多瓦大学的教授桑克托留斯（Sanctorius,1561-1636) 所设计的最早的体温计和脉搏计是根据伽利略的发明而加以改制的。

#### 1．生理学的进展

17世纪初，由于量度的应用，使生命科学的研究步入科学轨道。其标志之一是英国医学家威康•哈维（ Wiliam Harvey,1578-—1657） 发现血液循环，创建了血流循环学说。从而使生理学从解剖学中分立出来。哈维首先应用活体解剖的实验方法，并应用度量的概念，精确地计算出心脏每分钟搏出血量和每小时搏出血量。他于1628年发表了著作心血运动论）(The Movement of the heart and the Blood), 标志着血液循环理论的建立。恩格斯对哈维的发现做出这样的评价：“由于哈维发现血液循环，而把生理学确立为一门科学。”生理学家巴甫洛夫 (Ivan Pavlov,1849—1936）也评价说；“哈维的研究为动物生理学奠定了基础。”

**************** rank:2 *****************
显微镜的应用为 19 世纪细胞学的建立打下了良好的基础。

#### 3．医学三学派的成熟

在17世纪，由于物理学、化学和生物学的进步，使一些学者主张以单一学科的理论来解释生命现象和病理现象，出现三个学派。

（1）物理学派：代表人物是法国数学家、物理学家笛卡尔。他认为：“宇宙是一个庞大的机械，人的身体也是一部精细的机械，从宏观到微观，所有物体无一不可用机械原理来阐明。”身体的一切疼痛、恐怖表现都是机械的反应。伽利略的学生波累利认为肌肉运动是一种力学原理，人心脏的搏动、胃肠 运动都符合力学原理。他甚至认为胃的消化功能就是摩擦